## Imports

In [4]:
from geopy.geocoders import GoogleV3
import pandas as pd
import numpy as np
import os
import geopandas as gpd

## Read-in

In [5]:
df = pd.read_csv('Hudson_yards_map_template_popup_pin - Sheet1.csv')

In [6]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
project_gdf = gpd.read_file('hudson_yards.kml',driver = 'KML')
project_gdf['center_point'] = project_gdf.geometry.centroid

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_56485/407052935.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  project_gdf['center_point'] = project_gdf.geometry.centroid


## Geocoder Setup

In [7]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [8]:
df['geo_address'] = df['full_address'] + ' Manhattan, NY'
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## Correction Section

## HTML Popup Formatter

In [11]:
df.columns

Index(['full_address', 'sq_ft', 'developers', 'description', 'story_link',
       'project name', 'floors', 'status', 'notable tenants', 'geo_address',
       'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [12]:
def popup_html(row):
    i = row
    address = df['full_address'].iloc[i]
    developers = df['developers'].iloc[i]
    description = df['description'].iloc[i]
    project_name = df['project name'].iloc[i]
    floors = df['floors'].iloc[i]
    status = df['status'].iloc[i]
    notable_tenants = df['notable tenants'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    Address: <strong>{}'''.format(address) + '''</strong><br>
    Developer: <strong>{}'''.format(developers) + '''</strong><br>
    Description: <strong>{}'''.format(description) + '''</strong><br>
    Project Name: <strong>{}'''.format(project_name) + '''</strong><br>
    Floors: <strong>{}'''.format(floors) + '''</strong><br>
    Status: <strong>{}'''.format(status) + '''</strong><br>
    Notable Tenants: <strong>{}'''.format(notable_tenants) + '''</strong><br>
    </html>
    '''
    return html


In [14]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=13)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Hudson Yards Projects')

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True), min_width=300, max_width=300)
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)
    
for i, r in project_gdf.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j)
    geo_j.add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [30]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [28]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [29]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/hudson_yards_map_Pat_Ralph
